In [1]:
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy.ma as ma
from scipy.interpolate import griddata
from pyresample import geometry,image, kd_tree


In [2]:
# Path
respath='/data/jain1/c/sshu3/SBGC/results/SBGC_regional/'
contemp='2006_2015'
preind='1901_1910'
future_rcp45='rcp45_2091_2100'
future_rcp85='rcp85_2091_2100'

# Read the 3D files, For ISAM-1D
# Historical
fp=respath+preind+'/mean_bgc3d_'+preind+'.nc'
nclu_preind=NetCDFFile(fp,'r')
soclev_preind = nclu_preind.variables['soil_c_lev'][:]
resplev_preind = nclu_preind.variables['resp_lev'][:]
latnc = nclu_preind.variables['lat'][:]
lonnc = nclu_preind.variables['lon'][:]
#lonnc = ((lonnc - 180) % 360) - 180

# 20th century
fp=respath+contemp+'/mean_bgc3d_'+contemp+'.nc'
nclu_contemp=NetCDFFile(fp,'r')
soclev_contemp = nclu_contemp.variables['soil_c_lev'][:]
resplev_contemp = nclu_contemp.variables['resp_lev'][:]

# 21th century - RCP45
fp=respath+future_rcp45+'/mean_bgc3d_'+future_rcp45+'.nc'
nclu_rcp45=NetCDFFile(fp,'r')
soclev_rcp45 = nclu_rcp45.variables['soil_c_lev'][:]
resplev_rcp45 = nclu_rcp45.variables['resp_lev'][:]

# 21th century - RCP85
fp=respath+future_rcp85+'/mean_bgc3d_'+future_rcp85+'.nc'
nclu_rcp85=NetCDFFile(fp,'r')
soclev_rcp85 = nclu_rcp85.variables['soil_c_lev'][:]
resplev_rcp85 = nclu_rcp85.variables['resp_lev'][:]

# Path
respath='/data/jain1/c/sshu3/SBGC/results/SBGC_0d/'
contemp='2006_2015'
preind='1901_1910'
future_rcp45='rcp45_2091_2100'
future_rcp85='rcp85_2091_2100'

# Read the 3D files, For ISAM-0D
# Historical
fp=respath+preind+'/mean_bgc2d_'+preind+'.nc'
nclu_preind=NetCDFFile(fp,'r')
soc0d_preind = nclu_preind.variables['soil_c'][:]
resp0d_preind = nclu_preind.variables['resp'][:]
#lonnc = ((lonnc - 180) % 360) - 180

# 20th century
fp=respath+contemp+'/mean_bgc2d_'+contemp+'.nc'
nclu_contemp=NetCDFFile(fp,'r')
soc0d_contemp = nclu_contemp.variables['soil_c'][:]
resp0d_contemp = nclu_contemp.variables['resp'][:]

# 21th century - RCP45
fp=respath+future_rcp45+'/mean_bgc2d_'+future_rcp45+'.nc'
nclu_rcp45=NetCDFFile(fp,'r')
soc0d_rcp45 = nclu_rcp45.variables['soil_c'][:]
resp0d_rcp45 = nclu_rcp45.variables['resp'][:]

# 21th century - RCP85
fp=respath+future_rcp85+'/mean_bgc2d_'+future_rcp85+'.nc'
nclu_rcp85=NetCDFFile(fp,'r')
soc0d_rcp85 = nclu_rcp85.variables['soil_c'][:]
resp0d_rcp85 = nclu_rcp85.variables['resp'][:]

# Open NCSCD as observation
#path='/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/ncscdv2_soilc_0.5x0.5.nc'
path='/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/ncscd_masked_0.5x0.5.nc'
nclu_obs=NetCDFFile(path,'r')
soc_ncscd_orig = nclu_obs.variables['soilc'][:]
latnc2 = nclu_obs.variables['lat'][:]
lonnc2 = nclu_obs.variables['lon'][:]
soc_ncscd = soc_ncscd_orig[0,:,:]
#soc_ncscd[soc_ncscd<0] = float("nan")
#temp = soc_ncscd[:, 0:359]
#soc_ncscd[:, 0:359] = soc_ncscd[:, 360:719]
#soc_ncscd[:, 360:719] = temp[:,0:359]
#soc_ncscd[soc0d_contemp<0] = float("nan")

# Open the wetland fraction map to mask peatland 
respath = '/data/jain1/c/sshu3/SBGC/data/FW/'
fwet = 'min_fw_frac.nc'
fp=respath+fwet
nclu_wet=NetCDFFile(fp,'r')
wtld = nclu_wet.variables['FW'][:]
latnc3 = nclu_wet.variables['Lat'][:]
lonnc3 = nclu_wet.variables['Lon'][:]
wtld[wtld<0.05] = 0
wtld[wtld>=0.05] = 1
#wtld = np.flipud(wtld)

# Get the topsoil SOC (0-30cm) from 1D model
soc_top_preind = np.sum(soclev_preind[1:5,:,:], 0)
soc_top_contemp = np.sum(soclev_contemp[1:5,:,:], 0)
soc_top_rcp45 = np.sum(soclev_rcp45[1:5,:,:], 0)
soc_top_rcp85 = np.sum(soclev_rcp85[1:5,:,:], 0)
# topsoil resp
resp_top_preind = np.sum(resplev_preind[1:5,:,:], 0)
resp_top_contemp = np.sum(resplev_contemp[1:5,:,:], 0)
resp_top_rcp45 = np.sum(resplev_rcp45[1:5,:,:], 0)
resp_top_rcp85 = np.sum(resplev_rcp85[1:5,:,:], 0)
soc_top_rcp45
# Get the subsoil SOC (30-100cm) from 1D model
soc_sub_preind = np.sum(soclev_preind[5:7,:,:], 0)+0.3*soclev_preind[7,:,:]
soc_sub_contemp = np.sum(soclev_contemp[5:7,:,:], 0)+0.3*soclev_contemp[7,:,:]
soc_sub_rcp45 = np.sum(soclev_rcp45[5:7,:,:], 0)+0.3*soclev_rcp45[7,:,:]
soc_sub_rcp85 = np.sum(soclev_rcp85[5:7,:,:], 0)+0.3*soclev_rcp85[7,:,:]

# subsoil resp
resp_sub_preind = np.sum(resplev_preind[5:7,:,:], 0)+0.3*resplev_preind[7,:,:]
resp_sub_contemp = np.sum(resplev_contemp[5:7,:,:], 0)+0.3*resplev_contemp[7,:,:]
resp_sub_rcp45 = np.sum(resplev_rcp45[5:7,:,:], 0)+0.3*resplev_rcp45[7,:,:]
resp_sub_rcp85 = np.sum(resplev_rcp85[5:7,:,:], 0)+0.3*resplev_rcp85[7,:,:]

# Get the first 1m SOC
# 1D
soc_preind = soc_top_preind + soc_sub_preind
soc_contemp = soc_top_contemp + soc_sub_contemp
soc_rcp45 = soc_top_rcp45 + soc_sub_rcp45
soc_rcp85 = soc_top_rcp85 + soc_sub_rcp85

dsoc_rcp45 = soc_rcp45 - soc_contemp
dsoc_topsoil_rcp45 = soc_top_rcp45 - soc_top_contemp
dsoc_subsoil_rcp45 = soc_sub_rcp45 - soc_sub_contemp

dsoc_rcp85 = soc_rcp85 - soc_contemp
dsoc_topsoil_rcp85 = soc_top_rcp85 - soc_top_contemp
dsoc_subsoil_rcp85 = soc_sub_rcp85 - soc_sub_contemp

# 0D
dsoc0d_rcp45 = soc0d_rcp45 - soc0d_contemp
dsoc0d_rcp85 = soc0d_rcp85 - soc0d_contemp

# resp
# 1D
resp_preind = resp_top_preind + resp_sub_preind
resp_contemp = resp_top_preind + resp_sub_preind
resp_rcp45 = resp_top_rcp45 + resp_sub_rcp45
resp_rcp85 = resp_top_rcp85 + resp_sub_rcp85

dresp_rcp45 = resp_rcp45 - resp_contemp
dresp_rcp85 = resp_rcp85 - resp_contemp

dresp0d_rcp45 = resp0d_rcp45 - resp0d_contemp
dresp0d_rcp85 = resp0d_rcp85 - resp0d_contemp

# turnover, using annual resp/SOC pool
# Since it is tough to get the SOC D14C with ISAM when performing spinup, we right now don't use the D14C 
# for the calculation of the turnover for the regional results
# 1D
# First method, only get the total bulk turnover by using the total soc and soil resp
#tau_preind = soc_preind/resp_preind
#tau_contemp = soc_contemp/resp_contemp
#dtau = tau_contemp - tau_preind

# Second method, get the turnover time by calculating the turnover at each layer then calculate C werighted average.
# Get the turnover for each layer
tau_lev_preind = soclev_preind/resplev_preind
tau_lev_contemp = soclev_contemp/resplev_contemp
tau_lev_rcp45 = soclev_rcp45/resplev_rcp45
tau_lev_rcp85 = soclev_rcp85/resplev_rcp85


# Calculate the C mass weights
wt_preind = soclev_preind/soc_preind
wt_contemp = soclev_contemp/soc_contemp
wt_rcp45 = soclev_rcp45/soc_rcp45
wt_rcp85 = soclev_rcp85/soc_rcp85

tau_tmp_preind = wt_preind * tau_lev_preind
tau_tmp_contemp = wt_contemp * tau_lev_contemp
tau_tmp_rcp45 = wt_rcp45 * tau_lev_rcp45
tau_tmp_rcp85 = wt_rcp85 * tau_lev_rcp85

tau_preind = np.sum(tau_tmp_preind[1:7,:,:], 0)+0.3*tau_tmp_preind[7,:,:]
tau_contemp = np.sum(tau_tmp_contemp[1:7,:,:], 0)+0.3*tau_tmp_contemp[7,:,:]
tau_rcp45 = np.sum(tau_tmp_rcp45[1:7,:,:], 0)+0.3*tau_tmp_rcp45[7,:,:]
tau_rcp85 = np.sum(tau_tmp_rcp85[1:7,:,:], 0)+0.3*tau_tmp_rcp85[7,:,:]

dtau_rcp45 = tau_rcp45 - tau_contemp
dtau_rcp85 = tau_rcp85 - tau_contemp

# 0D
tau0d_preind = soc0d_preind/resp0d_preind
tau0d_contemp = soc0d_contemp/resp0d_contemp
tau0d_rcp45 = soc0d_rcp45/resp0d_rcp45
tau0d_rcp85 = soc0d_rcp45/resp0d_rcp85

dtau0d_rcp45 = tau0d_rcp45 - tau0d_contemp
dtau0d_rcp85 = tau0d_rcp85 - tau0d_contemp

# Also need to plot the ISAM-0D SOC results and NCSCD data to the right side 
# and compare against the mndel outputs
# Seems like the respiration output from 1D model has some problem
# Now let's check the 2D output
respath='/data/jain1/c/sshu3/SBGC/results/SBGC_regional/'
contemp='2006_2015'
preind='1901_1910'
# Read the 3D files, For ISAM-0D
# Historical
fp=respath+preind+'/mean_bgc2d_'+preind+'.nc'
nclu_preind=NetCDFFile(fp,'r')
resp_new_preind = nclu_preind.variables['resp'][:]
#lonnc = ((lonnc - 180) % 360) - 180
# Historical
fp=respath+contemp+'/mean_bgc2d_'+contemp+'.nc'
nclu_contemp=NetCDFFile(fp,'r')
resp_new_contemp = nclu_contemp.variables['resp'][:]

dresp_new = resp_new_contemp - resp_new_preind

In [ ]:
# Get the grid area
nlon = 720
nlat = 360
grid_area = np.ones((nlat, nlon))*-9999.
val = np.ones((nlat, nlon))*float("nan")
earth_area =  5.096e14;
lat = np.arange(-89.75, 90.25, 0.5)
res = 0.5;
for i in np.arange(0,nlat):
    for j in np.arange(0,nlon):
        grid_area[i,j] = (earth_area/2)*abs(np.sin((lat[i] - res/2)*np.pi/180) -
                    np.sin((lat[i] + res/2)*np.pi/180))/(360/res)
        val[i,j] = dtau_subsoil[i,j] * grid_area[i,j]
        


In [ ]:
pp/1e12

In [ ]:
# Get total SOC stock of the permafrost region
pp = np.nansum(val)

# Get the mean turnover time of the permafrost region



In [ ]:
tau0dnow = pp/(332.84*1e12)

In [ ]:
tau1dnow = pp/(288.72*1e12)

In [ ]:
tau1dnow

In [ ]:
ncscd = pp/1e12

In [ ]:
soc0dnow = pp/1e12

In [ ]:
soc1dnow = pp/1e12

In [ ]:
x, y = np.meshgrid(lonnc3, latnc3)
   
plt.figure(figsize=(9, 4))
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')

m.drawcoastlines(color='y', linewidth=0.5, linestyle='-')

nx = 720
ny = 360

# m.imshow(img_new, cmap='gist_gray')
m.scatter(x, y, s = wtld*0.05, latlon=True, marker='+', color='red')

plt.show()


In [8]:
# Plot the figure1
# Comparison of 1m SOC stock between two models and to the NCSCD data
fig = plt.figure(figsize=(30,36))

ax1 = fig.add_subplot(331)
ax1.set_title("ISAM-1D SOC stock (1m)",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
xs, ys = np.meshgrid(lonnc3, latnc3)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,dsoc_rcp45[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
# Overlap scatter plot to show the peatland distribution
#map.scatter(xs, ys, s = 0.4*wtld, latlon=True, marker='_', color='cyan')
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
#parallels = np.arange(45.,80.,15.)
## labels = [left,right,top,bottom]
#map.drawparallels(parallels)#,labels=[False,True,True,False])
#meridians = np.arange(15.,346.,30.)
#map.drawmeridians(meridians)#,labels=[True,False,False,True])

map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
#plt.title('Permafrost SOC stock')
cbar.ax.tick_params(labelsize=30) 

ax2 = fig.add_subplot(332)
ax2.set_title("ISAM-0D SOC stock (1m)",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_topsoil_rcp45[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
#map.scatter(xs, ys, s = 0.4*wtld, latlon=True, marker='_', color='cyan')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=30) 

lon2,lat2 = np.meshgrid(lonnc2,latnc2) #Returns coordinate matrices from coordinate vectors
x2,y2 = map(lon2,lat2)
ax2 = fig.add_subplot(333)
ax2.set_title("NCSCD SOC stock (1m)",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_subsoil_rcp45[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
                                                     # omit this to use linear
                                                     # color scale
#map.scatter(xs, ys, s = 0.4*wtld, latlon=True, marker='_', color='cyan')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=30) 

ax3 = fig.add_subplot(334)
ax3.set_title("ISAM-1D SOC turnover time",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_rcp85[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
#cbar.ax.set_xticklabels([0, 200, 400, 600, 800])  # horizontal colorbar
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 

ax3 = fig.add_subplot(335)
ax3.set_title("ISAM-0D SOC turnover time",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_topsoil_rcp85[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
#cbar.ax.set_xticklabels([0, 200, 400, 600, 800])  # horizontal colorbar
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 

ax3 = fig.add_subplot(336)
ax3.set_title("ISAM-0D SOC turnover time",fontsize=32)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc_subsoil_rcp85[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.BrBG_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
#cbar.ax.set_xticklabels([0, 200, 400, 600, 800])  # horizontal colorbar
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=30) 


#plt.show()
plt.savefig('./figures_seminar/dSOC_rcp.png',dpi=300)

In [ ]:
# Plot the figure7
# Comparison of 1m SOC stock between two models and to the NCSCD data
fig = plt.figure(figsize=(18,15))

#ax1 = fig.add_subplot(331)
ax1 = fig.add_subplot(221)
ax1.set_title("ISAM-1D SOC stock (2000s - 1900s)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,dsoc[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.RdYlGn) # use log color scale,
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
#plt.title('Permafrost SOC stock')
cbar.ax.tick_params(labelsize=22) 

ax2 = fig.add_subplot(222)
ax2.set_title("ISAM-0D SOC stock (2000s - 1900s)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dsoc0d[:,:],
              norm=colors.Normalize(vmin=-5.,vmax=5.),cmap=plt.cm.RdYlGn) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=22) 


ax3 = fig.add_subplot(223)
ax3.set_title("ISAM-1D SOC turnover time (2000s - 1900s)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dtau[:,:],
              norm=colors.Normalize(vmin=-60,vmax=60.),cmap=plt.cm.RdYlGn) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=22) 


ax4 = fig.add_subplot(224)
ax4.set_title("ISAM-0D SOC turnover time (2000s - 1900s)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,dtau0d[:,:],
              norm=colors.Normalize(vmin=-60.,vmax=60.),cmap=plt.cm.RdYlGn) # use log color scale,
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
map.drawparallels(np.arange(-80.,81.,20.))#, labels=[True,True,True,True])
map.drawmeridians(np.arange(-180.,181.,20.))#, labels=[True,True,True,True])
cbar.ax.tick_params(labelsize=22) 

#plt.show()
plt.savefig('./figures/fig7.png',dpi=300)

In [ ]:
# Plot the figure3
# Comparison of Soil respiration and the change
fig = plt.figure(figsize=(18,15))

ax1 = fig.add_subplot(221)
ax1.set_title("ISAM 0D Heterotrophic respiration (gC/m2/year)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
lon,lat = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
x,y = map(lon,lat)
clevs = np.power(10.0, np.arange(-1.0,2.5,0.5))
cs1 = map.pcolormesh(x,y,1000.*resp0d_contemp[:,:],
              norm=colors.Normalize(vmin=0.,vmax=500.),cmap=plt.cm.jet) # use log color scale,
cbar = map.colorbar(cs1,location='bottom',pad="5%",extend='both')
#plt.title('Permafrost SOC stock')
cbar.ax.tick_params(labelsize=22) 

ax2 = fig.add_subplot(222)
ax2.set_title("ISAM 0D change of Rh (gC/m2/year)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*dresp0d[:,:],
              norm=colors.Normalize(vmin=-50.,vmax=50.),cmap=plt.cm.RdYlGn_r) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 


ax3 = fig.add_subplot(223)
ax3.set_title("ISAM 1D Heterotrophic respiration (gC/m2/year)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*resp_new_contemp[:,:],
              norm=colors.Normalize(vmin=0.,vmax=500.),cmap=plt.cm.jet) # use log color scale,
                                                      # omit this to use linear
                                                     # color scale
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 


ax4 = fig.add_subplot(224)
ax4.set_title("ISAM 1D change of Rh (gC/m2/year)",fontsize=20)
map = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
# draw coastlines, state and country boundaries, edge of map.
map.drawcoastlines()
#map.drawstates()
#map.drawcountries(color='b')
cs = map.pcolormesh(x,y,1000.*dresp_new[:,:],
              norm=colors.Normalize(vmin=-50.,vmax=50.),cmap=plt.cm.RdYlGn_r) # use log color scale,
cbar = map.colorbar(cs,location='bottom',pad="5%",extend='both')
cbar.ax.tick_params(labelsize=22) 

plt.show()
#plt.savefig('Fig8.jpg',dpi=300)

In [ ]:
#http://matplotlib.org/basemap/users/mapsetup.html



In [ ]:
lonnc2

In [ ]:
lonnc2 % 360
